# __Hands-on with TensorFlow: Part B__

- In this demo, let's see how to implement ensemble techniques using Keras and sklearn.

## Step 1: Import the required libraries and load the dataset

- Import necessary libraries from Sklearn, TensorFlow, and Keras.
- Load the dataset using sklearn's make_moons function.
- Define X and y and split the dataset into training and testing sets using the train_test_split function.

In [ ]:
import sklearn
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

X,y=make_moons(n_samples=500,noise=0.30,random_state=42)

X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42)

__Observation:__
- The necessary libraries and modules have been downloaded.

## Step 2: Define classifiers

- Define Logistic Regression, Random Forest, Support Vector Machine, and Keras Neural Network classifiers.


In [ ]:
log_clf = LogisticRegression(solver="lbfgs", random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
svm_clf = SVC(gamma="scale", random_state=42,probability=True)


- Import Sequential and Dense modules from the TensorFlow library to create a neural network.
- Define the function build_nn() to build a neural network.


In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

def build_nn():
    model = Sequential([
        Dense(50, activation='relu', input_shape=[2]),
        Dense(1, activation='sigmoid')
    ])
    model.compile(
        optimizer='Adam',
        loss='binary_crossentropy',
        metrics=['accuracy'])
    return model

- Now, let's wrap scikit learn and build a neural network with 500 epochs with the Keras classifier.

In [ ]:
pip install scikeras

In [ ]:
import tensorflow as tf
from scikeras.wrappers import KerasClassifier
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

def build_nn():
    model = Sequential([
        Dense(50, activation='relu', input_shape=[2]),
        Dense(1, activation='sigmoid')
    ])
    model.compile(
        optimizer='Adam',
        loss='binary_crossentropy',
        metrics=['accuracy'])
    return model

keras_clf = KerasClassifier(build_fn=build_nn, epochs=500, verbose=False)

__Observation:__
- Scikit learn is wrapped.

In [ ]:
keras_clf._estimator_type = "classifier"

## Step 3: Create the voting classifier

- Combine the classifiers using sklearn's VotingClassifier with soft voting.


In [ ]:
voting = VotingClassifier(
             estimators=[('lr', log_clf),
                         ('rf', rnd_clf),
                         ('svc', svm_clf),
                         ('keras',keras_clf)],
             voting='soft',
             flatten_transform=True)

- The classifiers are combined with soft voting.

## Step 4: Train and evaluate the classifiers

- Train the voting classifier and each individual classifier.
- Make predictions on the test set and calculate the accuracy for each classifier.


In [ ]:
voting = VotingClassifier(estimators=[('Keras', KerasClassifier(model=build_nn, epochs=500, verbose=False))])
voting.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


VotingClassifier(estimators=[('Keras',
                              KerasClassifier(epochs=500, model=<function build_nn at 0x7f68e725c700>, verbose=False))])

- Consider voting with VotingClassifier and estimator.
- The first is LogisticRegression, the second is RandomForest, the third is SVM, the fourth is KerasClassifier, and the fifth is VotingClassifier.

In [ ]:
for clf in (log_clf, rnd_clf, svm_clf, keras_clf, voting):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.864
RandomForestClassifier 0.896
SVC 0.896


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


KerasClassifier 0.888


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


VotingClassifier 0.896


__Observation:__

- As shown above, LogisticRegression is at 86% accuracy, RandomForest is at 89%, SVM is at 89%, KerasClassifier is at 89%, and VotingClassifier is at 92%.